In [ ]:
pino_val_path = "D:/data/metric/all/val/pino"
pino_test_path = "D:/data/metric/all/test/pino"


res_val_path = "D:/data/metric/all/val/pino_opt"
res_test_path = "D:/data/metric/all/test/pino_opt"

csv_path = "D:/data/metric/all_csv"

In [ ]:
from rtmag.process.paper.utils import load_yaml_to_dict

config_dict = load_yaml_to_dict("metrics.yaml")

dataset_path = config_dict["dataset_path"]
exc_noaa = config_dict["exc_noaa"]
val_path = config_dict["val_path"]
test_path = config_dict["test_path"]

In [ ]:
from pathlib import Path

In [ ]:
files = list(Path(dataset_path).glob('**/label/*.npz'))
if isinstance(exc_noaa, str):
    files = sorted([f for f in files if not exc_noaa == str(f.parent.parent.stem)])
elif isinstance(exc_noaa, list):
    files = sorted([f for f in files if not exc_noaa[0] == str(f.parent.parent.stem)])
    for t_noaa in exc_noaa[1:]:
        files = sorted([f for f in files if not t_noaa == str(f.parent.parent.stem)])

In [ ]:
val_files = []
for d_path in val_path:
    val_files += list(Path(d_path).glob('**/label/*.npz'))
val_files = sorted([f for f in val_files])

In [ ]:
test_files = []
for d_path in test_path:
    test_files += list(Path(d_path).glob('**/label/*.npz'))
test_files = sorted([f for f in test_files])

In [ ]:
len(files), len(val_files), len(test_files)

In [ ]:
import numpy as np
from tqdm import tqdm

from rtmag.process.paper.utils import get_label
from rtmag.process.paper.opt_jit import relax_noprint

In [ ]:
pino_val_path = Path(pino_val_path)

res_val_path = Path(res_val_path)
res_val_path.mkdir(exist_ok=True, parents=True)

In [ ]:
from time import perf_counter

In [ ]:
for file in tqdm(val_files):
    inputs = pino_val_path / f"{file.stem}_pino.npy"
    targets = res_val_path / f"{file.stem}_pino_relax.npy"
    if not targets.exists():
        b_isee = get_label(file)
        bottom = b_isee[:, :, 0, :]

        b_pino = np.load(inputs)
        b_pino[:, :, 0, :] = bottom

        start_time = perf_counter()
        br = relax_noprint(b_pino, res_val_path / f"{file.stem}")
        end_time = perf_counter()
        total_time = end_time - start_time

        np.save(targets, br)

        with open(res_val_path / f"{file.stem}.txt", "w") as f:
            print(f"total_time: {total_time} seconds", file=f)

In [ ]:
pino_test_path = Path(pino_test_path)

res_test_path = Path(res_test_path)
res_test_path.mkdir(exist_ok=True, parents=True)

In [ ]:
for file in tqdm(test_files):    
    inputs = pino_test_path / f"{file.stem}_pino.npy"
    targets = res_test_path / f"{file.stem}_pino_relax.npy"
    if not targets.exists():
        b_isee = get_label(file)
        bottom = b_isee[:, :, 0, :]

        b_pino = np.load(inputs)
        b_pino[:, :, 0, :] = bottom

        start_time = perf_counter()
        br = relax_noprint(b_pino, res_test_path / f"{file.stem}")
        end_time = perf_counter()
        total_time = end_time - start_time
        
        np.save(targets, br)

        with open(res_test_path / f"{file.stem}.txt", "w") as f:
            print(f"total_time: {total_time} seconds", file=f)

In [ ]:
import pandas as pd
import rtmag.process.paper.metric as m

In [ ]:
csv_path = Path(csv_path)
csv_path.mkdir(exist_ok=True, parents=True)

In [ ]:
results = []
for file in tqdm(val_files):
    b_isee = get_label(file)

    target = res_val_path / f"{file.stem}_pino_relax.npy"
    br = np.load(target)

    res = m.evaluate(br, b_isee, ret=False)
    res.update({'file':file.stem})
    results.append(res)

    df = pd.DataFrame.from_dict(results)
    df.to_csv(csv_path / 'metrics_val_pino_opt.csv', index=False)

In [ ]:
results = []
for file in tqdm(test_files):
    b_isee = get_label(file)

    target = res_test_path / f"{file.stem}_pino_relax.npy"
    b_pot = np.load(target)

    res = m.evaluate(b_pot, b_isee, ret=False)
    res.update({'file':file.stem})
    results.append(res)

    df = pd.DataFrame.from_dict(results)
    df.to_csv(csv_path / 'metrics_test_pino_opt.csv', index=False)